# Arctic

> Arctic helper scripts and functions.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp arctic

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import os

import click
from arcticdb import Arctic, LibraryOptions
import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from pathlib import Path
from lobster_tools.config import MainConfig, register_configs
from lobster_tools.preprocessing import Data, Lobster
import sys
import pandas as pd

In [ ]:
# | export

# access config by normal python import
# cfg = MainConfig()
# register configs and then build object
register_configs()

with initialize(version_base=None, config_path=None):
    cfg_omega = compose(config_name="config")
    cfg: MainConfig = OmegaConf.to_object(compose(config_name="config"))
    # print(cfg)

MainConfig(hydra=CustomHydraConf(defaults=[{'output': 'default'}, {'launcher': 'basic'}, {'sweeper': 'basic'}, {'help': 'default'}, {'hydra_help': 'default'}, {'hydra_logging': 'default'}, {'job_logging': 'default'}, {'callbacks': None}, {'env': 'default'}], mode=None, searchpath=[], run=RunDir(dir='../custom_directory/${now:%Y-%m-%d}/${now:%H-%M-%S}'), sweep=SweepDir(dir='???', subdir='???'), hydra_logging='???', job_logging='???', sweeper='???', launcher='???', callbacks={}, help=HelpConf(app_name='???', header='???', footer='???', template='???'), hydra_help=HydraHelpConf(hydra_help='???', template='???'), output_subdir='.hydra', overrides=OverridesConf(hydra=[], task=[]), job=JobConf(name='???', chdir=None, override_dirname='???', id='???', num='???', config_name='???', env_set={}, env_copy=[], config=JobConf.JobConfig(override_dirname=JobConf.JobConfig.OverrideDirname(kv_sep='=', item_sep=',', exclude_keys=[]))), runtime=RuntimeConf(version='???', version_base='???', cwd='???', co

In [ ]:
# | export
# register_configs()
# @hydra.main(version_base=None, config_name="config")
# def test_arctic_db(cfg: MainConfig) -> None:
#     """Connect to arctic db and list symbols in the `lobster` library."""
#     # ARCTIC_DATABASE_PATH = '/nfs/home/nicolasp/home/data/arctic'
#     conn = f'lmdb://{cfg.db.db_path}'
#     arctic = Arctic(conn)
#     print(arctic.list_libraries())

#     library = arctic['lobster']
#     print(library.list_symbols())

In [ ]:
# | exports
# | code-fold: true

register_configs()
CONTEXT_SETTINGS = dict(
    help_option_names=["-h", "--help"],
    token_normalize_func=lambda x: x.lower() if isinstance(x, str) else x,
    show_default=True,
)

@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-p", "--path", default=cfg.db.db_path, help="database path")
# @click.option("-l", "--levels", default=cfg.sample_data.levels, help="number of levels")
def test_arctic_db(path
):
    """Connect to arctic db and list symbols in the `lobster` library."""
    conn = f'lmdb://{path}'
    arctic = Arctic(conn)
    print(arctic.list_libraries())

    library = arctic['lobster']
    print(library.list_symbols())


In [ ]:
# | exports
# | code-fold: true

@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-t", "--ticker", required=True, help="ticker to print")
@click.option("-s", "--start_date", default="2020-01-01", help="start date")
@click.option("-e", "--end_date", default="2020-02-01", help="end date")
# @click.option("-l", "--levels", default=cfg.sample_data.levels, help="number of levels")
def test_arctic_read_symbol(db_path, ticker, start_date, end_date,
):
    """Connect to arctic db and list symbols in the `lobster` library."""
    date_range = (start_date, end_date)
    conn = f'lmdb://{db_path}'
    arctic = Arctic(conn)

    library = arctic['lobster']
    print('available tickers:')
    print(library.list_symbols())
    print("start date and end date not used yet")
    df = library.read(ticker).data
    
    print(f"Printing df.head() for ticker {ticker}")
    print(df.head())

    print("columns of df are:")
    print(df.columns)

In [ ]:
# | exports
# | code-fold: true

@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path")
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-t", "--ticker", required=True, help="ticker to write to db")
@click.option("-s", "--start_date", default="2020-01-01", help="start date")
@click.option("-e", "--end_date", default="2020-02-01", help="end date")
# @click.option("-l", "--levels", default=cfg.sample_data.levels, help="number of levels")
def test_arctic_write_symbol(db_path, csv_path, ticker, start_date, end_date,
):
    conn = f'lmdb://{db_path}'
    arctic = Arctic(conn)
    print(arctic)
    library = arctic['lobster']

    date_range = (start_date, end_date)
    
    if ticker in library.list_symbols():
        print('warning - think about this in a second')
        user_input = input("Are you sure you want to overwrite this symbol? (y/n):")
        user_input = user_input.lower()
        if user_input == "y":
            # no need to delete. it's versioned
            pass
            # library.delete(symbol=ticker)
            # print("finished deleting")
        elif user_input == "n":
            print("Deletion canceled.")
            sys.exit(0)
        else:
            print("Invalid input. Deletion canceled.")
            sys.exit(1)

    print("loading data:")
    data = Data(directory_path=csv_path, ticker=ticker, date_range=date_range, aggregate_duplicates=False)
    lobster = Lobster(data=data)
    df = pd.concat([lobster.messages, lobster.book], axis=1)
    print('finished preprocessing')
    
    library.write(symbol=ticker, data=df)

In [ ]:

# connection to arctic-db
CSV_DATA_PATH="/nfs/home/nicolasp/home/data/tmp"
ARCTIC_DATABASE_PATH = '/nfs/home/nicolasp/home/data/arctic'
conn = f'lmdb://{ARCTIC_DATABASE_PATH}'
arctic = Arctic(conn)
library = arctic['lobster']

FLAGS = flags.FLAGS
flags.DEFINE_string("ticker", None, "ticker to convert to arctic db")
flags.mark_flag_as_required("ticker")

flags.DEFINE_string("start_date", default="2020-01-01", help="keep all files from start_date until end_date")
flags.DEFINE_string("end_date", default="2020-02-01", help="keep all files from start_date until end_date")

def main(argv):
    ticker = FLAGS.ticker
    print(f"ticker is {ticker}")
    date_range = (FLAGS.start_date, FLAGS.end_date)
    
    if ticker in library.list_symbols():
        print('warning - think about this in a second')
        user_input = input("Are you sure you want to overwrite this symbol? (y/n):")
        user_input = user_input.lower()
        if user_input == "y":
            pass
        elif user_input == "n":
            print("Deletion canceled.")
            sys.exit(0)
        else:
            print("Invalid input. Deletion canceled.")
            sys.exit(1)

        data = Data(directory_path=CSV_DATA_PATH, ticker=ticker, date_range=date_range, aggregate_duplicates=False)
        lobster = Lobster(data=data)
        df = pd.concat([lobster.messages, lobster.book], axis=1)
        
        # symbol still shows up when running ./test-db-connection.py
        # library.delete(symbol=ticker)
        
        library.write(symbol=ticker, data=df)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()